In [ ]:
##1. Random Sampling: Each of you will use a random sample of 10K instances drawn from the 61K instances in the secondary mushroom data. Briefly explain the random sampling code cell(s) in your submission notebook. [4 marks]
##2. Exploratory Data Analysis (EDA): Create appropriate visualisations to explore your dataset and summarise your findings about the data. Highlight your findings relevant to the model fitting stage (task 3 onwards). [10 marks]
##3. Model Shortlisting based on EDA: Based on your findings from the above EDA task, shortlist three classifiers from the classifiers you learnt in the lab classes. Explain your choice of classifiers in terms of your findings from the above EDA task. [5 marks]
##4. Model Fitting: Fit the chosen three classifiers to your sample of data, briefly explaining your choices and assumptions. [10 marks]
##5. Model Evaluation & model selection: Evaluate your three classifiers from the above task using the cross-validation method and explain the performance of your three classifiers. Explain how you use the cross-validation results to select the ‘winning’ classifier among the three. [15 marks]
##6. Final model Selection: In your lab classes, you learnt several classifiers, more than the three you selected in task 3 above. Now fit the remaining classifiers you learnt (excluding the three you already fitted in task 4) and evaluate all the new models fitted in this task using the cross- validation method. Explain how you use the cross-validation results from tasks 5 and 6 to select the final ‘winning’ classifier for your dataset. [15 marks]
##7. You have two ‘winning’ classifiers from tasks 5 and 6. Using the evaluation data from tasks 5 and 6 and your findings from the EDA in task 2, explain how helpful your EDA findings have been in improving the efficiency of the model fitting and selection process. [10 marks]
##8. Explain the top three lessons/insights you gained from tasks 1 to 6 in building classification models. [6 marks]